In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-131812")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131812
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-131812


In [59]:
!python train.py --C=0.5 --max_iter=10

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pyOpenSSL 20.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyOpenSSL 20.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 20.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:St

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "newcompute5567"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [60]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
### YOUR CODE HERE ### 
parameter_space={
    "C": uniform(0,1),
    "max_iter" : choice(10,20,50,100,150)
}
ps =RandomParameterSampling(parameter_space, properties=None)

# Specify a Policy
highpolicy = BanditPolicy(slack_factor=0.3)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
from azureml.core.script_run_config import ScriptRunConfig
est = ScriptRunConfig(source_directory="./", script= "train.py", compute_target=cpu_cluster )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name="accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4, 
                                     policy=highpolicy, 
                                     estimator=est)

In [61]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.train.hyperdrive import HyperDriveRun
HPrun = HyperDriveRun(exp,run_id=run.id, hyperdrive_config=hyperdrive_config)
# RunDetails(HPrun).show()
HPrun.get_metrics()
#HPrun.log("accuracy", 0.95)

{}

In [62]:
import joblib
# Get your best run and save the model from that run.
best_run = HPrun.get_best_run_by_primary_metric()
joblib.dump(best_run, 'hdRun_1.joblib') 
### YOUR CODE HERE ###

['hdRun_1.joblib']

In [63]:
print(best_run)

None


In [31]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
webpath = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(webpath)

In [33]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data= pd.concat([x_train.reset_index(drop=True), y_train], axis=1)

NameError: name 'clean_data' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name=y_train,
    n_cross_validations=4)

In [2]:
# Submit your automl run
### YOUR CODE HERE ###
from azureml.train.automl.run import AutoMLRun
automl_run = AutoMLRun(exp, run_id = 'AutoML_9f-7746')

RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###
best_run_aml = automl_run.get_best_child()
joblib.dump(best_run_aml, 'amlRun_1.joblib') 